In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

tf.random.set_seed(1323)

tf_df = pd.read_parquet('data/transformed.parquet')
# tf_df = pd.read_parquet('data/nottransformed.parquet')

tf_df.head(10).T

,0,1,2,3,4,5,6,7,8,9
STATUS,1,1,1,1,1,1,1,1,1,1
SPECIAL_CONSIDERATIONS,0,0,0,0,0,0,0,0,0,0
IS_SUCCESSFUL,1,1,0,1,1,1,1,1,1,0
AFFILIATION_CompanySponsored,0,0,1,1,0,0,0,0,0,1
AFFILIATION_Family/Parent,0,0,0,0,0,0,0,0,0,0
AFFILIATION_Independent,1,1,0,0,1,1,1,1,1,0
AFFILIATION_National,0,0,0,0,0,0,0,0,0,0
AFFILIATION_Other,0,0,0,0,0,0,0,0,0,0
AFFILIATION_Regional,0,0,0,0,0,0,0,0,0,0
APPLICATION_TYPE_T10,1,0,0,0,0,0,0,0,0,0


In [3]:
# this time we'll just scale ASK_AMT

tf_df.head()

,STATUS,SPECIAL_CONSIDERATIONS,IS_SUCCESSFUL,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,APPLICATION_TYPE_T10,...,INCOME_AMT_5M-10M,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev
0,1,0,1,0,0,1,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1
1,1,0,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2,1,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
3,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
4,1,0,1,0,0,1,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0


In [4]:
tf_df.shape

(34299, 53)

In [5]:
# Split our preprocessed data into our features and target arrays
y = tf_df['IS_SUCCESSFUL'].values
X = tf_df.drop(columns='IS_SUCCESSFUL').values


In [6]:

# Define a range of hyperparameters to search over
learning_rates = [0.0005, 0.001, 0.002]
batch_sizes = [10, 20, 30]
layer_configs = [(32, 16), (16,16), (16,8)]  # Varying the units in each layer
activation_functions = ['relu', 'swish', 'LeakyReLU']

best_accuracy = 0
best_hyperparameters = {}


# in four nested loops, we test each variation of the above
# the final nested loop performs k-fold cross-validation, k=4

kf = StratifiedKFold(n_splits=4, shuffle=True, random_state=132)

n_features = len(tf_df.columns) - 1
iteration = 0
total_iterations = len(learning_rates) * len(batch_sizes) * len(layer_configs) * len(activation_functions)

for learning_rate in learning_rates:
    for batch_size in batch_sizes:
        for layer_config in layer_configs:
            for activation_func in activation_functions:
                iteration += 1
                print(f"Iteration {iteration}/{total_iterations}: Learning Rate = {learning_rate}, Batch Size = {batch_size}, Layer Config = {layer_config}, Activation = {activation_func}")

                val_accuracies = []

                for train_idx, val_idx in kf.split(X, y):
                    nn = tf.keras.models.Sequential()

                    # Adding variable layers and units
                    nn.add(Dense(units=layer_config[0], activation=activation_func, input_dim=n_features))
                    nn.add(Dense(units=layer_config[1], activation=activation_func))
                    
                    # Output layer
                    nn.add(Dense(units=1, activation='sigmoid'))

                    nn.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate), metrics=['accuracy'])

                    X_fold, X_val_fold = X[train_idx], X[val_idx]
                    y_fold, y_val_fold = y[train_idx], y[val_idx]

                    early_stopping = EarlyStopping(monitor='val_accuracy', patience=10)

                    fit_model = nn.fit(X_fold, y_fold, epochs=50, batch_size=batch_size, verbose=0, validation_data=(X_val_fold, y_val_fold), callbacks=[early_stopping])

                    _, val_accuracy = nn.evaluate(X_val_fold, y_val_fold, verbose=0)
                    val_accuracies.append(val_accuracy)

                avg_val_accuracy = np.mean(val_accuracies)

                if avg_val_accuracy > best_accuracy:
                    best_accuracy = avg_val_accuracy
                    best_hyperparameters = {'learning_rate': learning_rate, 'batch_size': batch_size, 'layer_config': layer_config, 'activation_func': activation_func}

print("Best Hyperparameters:", best_hyperparameters)
print("Best Validation Accuracy:", best_accuracy)


Iteration 1/81: Learning Rate = 0.0005, Batch Size = 10, Layer Config = (32, 16), Activation = relu
Iteration 2/81: Learning Rate = 0.0005, Batch Size = 10, Layer Config = (32, 16), Activation = swish
Iteration 3/81: Learning Rate = 0.0005, Batch Size = 10, Layer Config = (32, 16), Activation = LeakyReLU
Iteration 4/81: Learning Rate = 0.0005, Batch Size = 10, Layer Config = (16, 16), Activation = relu
Iteration 5/81: Learning Rate = 0.0005, Batch Size = 10, Layer Config = (16, 16), Activation = swish
Iteration 6/81: Learning Rate = 0.0005, Batch Size = 10, Layer Config = (16, 16), Activation = LeakyReLU
Iteration 7/81: Learning Rate = 0.0005, Batch Size = 10, Layer Config = (16, 8), Activation = relu
Iteration 8/81: Learning Rate = 0.0005, Batch Size = 10, Layer Config = (16, 8), Activation = swish
Iteration 9/81: Learning Rate = 0.0005, Batch Size = 10, Layer Config = (16, 8), Activation = LeakyReLU
Iteration 10/81: Learning Rate = 0.0005, Batch Size = 20, Layer Config = (32, 16), Ac